In [ ]:
# This jupyter notebook presents a few simple examples of how to use the code PyDislocDyn as a module;
# Please look at the manual for a more in-depth documentation, as well as the doc-strings
# of the (sub-)module(s), classes, and functions for those options not covered by the manual
import pydislocdyn
pydislocdyn.writeinputfile("Cu",fname="Cu.in") # default: fname equals 1st argument
# pydislocdyn.writeallinputfiles() # generates many inputfiles from pydislocdyn.metal_data (calls writeinputfile() for each one)

In [ ]:
Cu = pydislocdyn.readinputfile("Cu.in") # optional: theta=array of character angles (default is [0,pi/2])
Cu

In [ ]:
Cu.theta

In [ ]:
# compute isotropic averages of single crystal elastic constants (assuming no texturing):
Cu.compute_Lame(include_TOEC=True,scheme='hill') ## default: scheme='auto' uses Kroeners average for SOEC

In [ ]:
# remark: for an isotropic solid at 2nd order, only 2 elastic constants are independent.
# this function determines all other commonly used ones from any 2:
pydislocdyn.convert_SOECiso(bulk=130,poisson=0.3)

In [ ]:
# compare measures of anisotropy, i.e. Zener's ratio (for cubic crystals) and the universal log-Euclidean anisotropy index
print(f"{Cu.Zener}, {Cu.anisotropy_index()}")

In [ ]:
# determine the limiting velocities in m/s of all dislocation characters initialized in the class-instance:
Cu.computevcrit(return_all=True) ## default: return_all=False; only returns branch 0 as numpy array

In [ ]:
# regardless of 'return_all' keyword, results are stored in the .vcrit_all attribute:
Cu.vcrit_all[0],Cu.vcrit_all[1:]

In [ ]:
Cu.findvcrit_smallest() ## find the lowest limiting velocity in m/s for all dislocation character angles of a given slip system
print(f"{Cu.vcrit_smallest}, {Cu.vcrit_edge}, {Cu.vcrit_screw}")

In [ ]:
# compute dislocation drag from phonon wind for gliding velocities beta and character angles theta:
pydislocdyn.phonondrag(Cu,beta=[0.01,0.2,0.5,0.69]) ## beta = v/Cu.ct; units: [B]=mPa s

In [ ]:
# now let's visualize the displacement gradient field of one component of a dislocation gliding at velocity beta (1st argument)
# defaults: character='screw',component=[2,0],showplt=False,savefige=True
# Note: if LaTeX is found and we're not running in a jupyter notebook (or similar environment loading 'ipykernel'), 
# matplotlib's pgf backend is used with LaTeX to produce nicer pdf figures
Cu.plotdisloc(0.6,character='edge',component=[1,0],showplt=True,savefig=False)

In [ ]:
Cu.plotdisloc(0.8,a=1e14,showplt=True,savefig=False) ## accelerating screw disloc. at time where v(t)=0.8*Cu.ct=1857m/s

In [ ]:
Cu.findRayleigh() # for character angles Cu.theta

In [ ]:
# find "radiation-free" transonic gliding velocities; note these are specific to "perfect" dislocations
Cu.find_vRF() # (no radiation free velocities for this metal)

In [ ]:
Cu.computesound([1,1,1]) # find the sound speeds for a given direction of propagation in the crystal

In [ ]:
# find the lowest (which='l'=default) and highest (which='h') sound speed, as well as the highest (quasi-)shear wave speed (which='hs'),
# in the crystal.
Cu.find_wavespeed(accuracy=0.01),Cu.find_wavespeed(which='hs',accuracy=0.01,maxfun=1200), Cu.find_wavespeed(which='h',accuracy=0.01)

In [ ]:
import numpy as np
## for line tension calcs, need to initialize with many character angles:
Cu_mixed = pydislocdyn.readinputfile("Cu.in",Ntheta=250,include_extra=True)
print(len(Cu_mixed.theta),Cu_mixed.theta[[0,1,-2,-1]]/np.pi) ## kw include_extra adds two character angles at the edges

In [ ]:
Cu_mixed.computeuij(0.6) # v=0.6*ct
Cu_mixed.computeEtot()
Cu_mixed.computeLT()
Cu_mixed.LT[0],Cu_mixed.LT[-1],len(Cu_mixed.LT) ## linetension needs two derivatives wrt theta, these are the screw/edge results:

In [ ]:
# some of the computations above also work for symbolic expressions, for example:
# find Voigt and Reuss averages for general crystal at the example of hcp:
import sympy as sp
hcpcryst = pydislocdyn.metal_props(sym='hcp')
# hcpcryst.c11,hcpcryst.c12,hcpcryst.c13,hcpcryst.c33,hcpcryst.c44=sp.symbols('c11,c12,c13,c33,c44',real=True)
# hcpcryst.init_C2()
## define symbols for elastic constants manually (above) or use pre-configured set (below)
hcpcryst.init_symbols()
hcpcryst.C2

In [ ]:
hcpcryst.voigt = hcpcryst.compute_Lame(scheme='voigt')
hcpcryst.voigt

In [ ]:
# hcpcryst.rho = sp.symbols('ρ',positive=True) # already set by .init_symbols() method above
hcpcryst.init_sound()
sp.simplify(hcpcryst.ct/hcpcryst.cl)

In [ ]:
import pandas as pd
pd.set_option('display.max_colwidth', 70) ## show at least the bulk modulus without truncating 
hcpcryst.reuss = hcpcryst.compute_Lame(scheme='reuss')
hcpcryst.reuss

In [ ]:
sp.Matrix(hcpcryst.computesound([1,1,0]))

In [ ]:
sp.simplify(hcpcryst.anisotropy_index())

In [ ]:
## limiting velocities of dislocations are implemented for symbolic computation only for simple cases, e.g.:
Cu.init_symbols()
vcrit_fcc = Cu.computevcrit()

In [ ]:
vcrit_fcc['screw'] ## caveat: numerical prefactor sqrt(3/2) was converted to float

In [ ]:
vcrit_fcc['edge'] ## the 3 branches are not ordered since we can't know which one is smallest

In [ ]:
poly=pydislocdyn.strain_poly()
poly

In [ ]:
poly.generate_poly([poly.y,0,0,0,0,0],preserve_volume=False,P=0)

In [ ]:
poly.alpha, pydislocdyn.Voigt(poly.strain)